In [4]:
!pip install -q numpy
!pip install -q pandas
!pip install -q matplotlib

In [63]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import itertools

from utils.helpers import reading


In [73]:
data = './data/data.csv'
df = reading(data)



df.drop('Cost/Energy…?', axis=1, inplace=True)

# Convert the dataframe to a numpy array
df = df.astype(float)
df['Energy'] = df.sum(axis=1)
df.reset_index().set_index('Processes').head()



print(df.head(3))


0          M-001  M-002  M-003  M-004  M-005  M-006  M-007  M-008  M-009  \
Processes                                                                  
P-001      -73.0    0.0   59.0    0.0   48.0    0.0    0.0    0.0    0.0   
P-002        0.0    0.0    0.0    0.0   73.0    0.0    0.0   11.0    0.0   
P-003        0.0    0.0    0.0   58.0   26.0    0.0    0.0   92.0   43.0   

0          M-010  ...  M-012  M-013  M-014  M-015  M-016  M-017  M-018  M-019  \
Processes         ...                                                           
P-001        0.0  ...  -60.0    0.0    0.0  -26.0    0.0    0.0   -9.0  -15.0   
P-002       69.0  ...    0.0   36.0  -33.0   33.0  -82.0   64.0    0.0  -38.0   
P-003        0.0  ...  -42.0   34.0   -4.0    5.0    0.0    0.0    0.0    0.0   

0          M-020  Energy  
Processes                 
P-001        0.0   -76.0  
P-002      -92.0   112.0  
P-003        0.0   212.0  

[3 rows x 21 columns]


In [69]:
def process(df):
    l = []
    for process in df.index:
        for material in df.columns:
            l.append((process, material, df.loc[process, material]))

In [71]:

o = process(df)
print(o)

None
